In [ ]:
!pip install langchain
!pip install langchain-community
!pip install openai
!pip install gradio
!pip install huggingface_hub

In [ ]:
import os
import gradio as gr
from langchain.chat_models import ChatOpenAI
from langchain import LLMChain, PromptTemplate
from langchain.memory import ConversationBufferMemory

In [19]:
OPENAI_API_KEY="insert your OpenAI API key here"
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [ ]:
template = """You are a sports-loving high school student with a keen interest in multiple sports, from soccer and basketball to tennis and swimming. You closely follow sports events, stats, and news, making you the go-to person for all sports-related discussions and predictions.
{chat_history}
User: {user_message}
Chatbot:"""

prompt = PromptTemplate(
    input_variables=["chat_history", "user_message"], template=template
)

memory = ConversationBufferMemory(memory_key="chat_history")

In [ ]:
# from langchain.llms import HuggingFacePipeline
# hf = HuggingFacePipeline.from_model_id(
#     model_id="gpt2",
#     task="text-generation",)

In [ ]:
llm_chain = LLMChain(
    llm=ChatOpenAI(temperature='0.5', model_name="gpt-3.5-turbo"),
    prompt=prompt,
    verbose=True,
    memory=memory,
)

In [ ]:
def get_text_response(user_message,history):
    response = llm_chain.predict(user_message = user_message)
    return response

In [ ]:
demo = gr.ChatInterface(get_text_response, examples=["How are you doing?","What are your interests?","Which places do you like to visit?"])

In [ ]:
if __name__ == "__main__":
    demo.launch() #To create a public link, set `share=True` in `launch()`. To enable errors and logs, set `debug=True` in `launch()`.

**Publishing code to Hugging Face**

In [ ]:
import requests
import gradio as gr

PERPLEXITY_API_KEY = "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"  # your Perplexity key

def call_perplexity_api(user_message, chat_history):
    api_url = "https://api.perplexity.ai/chat"  # hypothetical endpoint, replace with real one
    headers = {
        "Authorization": f"Bearer {PERPLEXITY_API_KEY}",
        "Content-Type": "application/json",
    }
    # Construct the payload based on Perplexity API specs
    payload = {
        "messages": [
            {"role": "system", "content": "You are a sports-loving high school student..."},
            # Add previous messages to simulate chat history, if supported
            {"role": "user", "content": user_message},
        ]
    }

    response = requests.post(api_url, json=payload, headers=headers)
    if response.status_code == 200:
        data = response.json()
        # Extract the reply from Perplexity's response format
        answer = data.get("answer") or data.get("response") or data.get("content") or ""
        return answer
    else:
        return f"Error: {response.status_code} - {response.text}"

def chat(user_message, history):
    # Here you can process history if needed (append, shorten, etc.)
    # For now, just pass user_message forward
    return call_perplexity_api(user_message, history)

demo = gr.ChatInterface(chat)

if __name__ == "__main__":
    demo.launch(share=True)


In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
from huggingface_hub import HfApi
api = HfApi()

In [ ]:
HUGGING_FACE_REPO_ID = "Sumanth-13/GenAIChatBot"

In [ ]:
%mkdir /content/ChatBotWithOpenAI
!wget -P  /content/ChatBotWithOpenAI/ https://s3.ap-south-1.amazonaws.com/cdn1.ccbp.in/GenAI-Workshop/ChatBotWithOpenAIAndLangChain/app.py
!wget -P /content/ChatBotWithOpenAI/ https://s3.ap-south-1.amazonaws.com/cdn1.ccbp.in/GenAI-Workshop/ChatBotWithOpenAIAndLangChain/requirements.txt

In [ ]:
%cd /content/ChatBotWithOpenAI

api.upload_file(
    path_or_fileobj="./requirements.txt",
    path_in_repo="requirements.txt",
    repo_id=HUGGING_FACE_REPO_ID,
    repo_type="space")

api.upload_file(
    path_or_fileobj="./app.py",
    path_in_repo="app.py",
    repo_id=HUGGING_FACE_REPO_ID,
    repo_type="space")